In [86]:
import numpy as np
import os

### Euler

Simple single trajectory in a harmonic potential centered at 0 run for 100 time steps.

In [125]:
params = {
    "SpringConstant":1,
    "xo":0,
    # In case in later exercise we want to track energies.
    "ComputeEnergy":True,    
    "PotentialFunction":potentialHarmonic
}

Z = [1+0j]
Z = np.array(Z)
Qtraj = np.zeros(100)

# Short 100 step 'md' run.
for i in range(0, 100):
    Qtraj[i] = Z.real[0]
    Z = Euler(Z, forcesHarmonic, params)

# Checking to see if trajectory looks reasonable for single particle.
f = open("EulerTrajectory.xyz", "w+")
for i in Qtraj:
    f.write("1\nEulerStep\n")
    f.write("H\t"+ str(i)+ "\t0\t0\n")
f.close()

### Leap Frog

Another single trajectory in harmonic potential centered at 0 for 100 time steps.

In [126]:
params = {
    "SpringConstant":1,
    "xo":0,
    #These two lines in case we end up having to track energies later on, does nothing for now.
    "ComputeEnergy":True,    
    "PotentialFunction":potentialHarmonic
}

#Initialize the previous coordinates to be simply q_i(t-1) = .95q_i(t)
Y = np.array([1+0j])
Z = np.array([Y, .95*Y])


Qtraj = np.zeros(100)
# Short 100 step 'md' run.
for i in range(0, 100):
    Qtraj[i] = Z[0].real[0]
    Z = LeapFrog(Z, forcesHarmonic, params)

# Checking to see if trajectory looks reasonable for single particle.
f = open("LeapFrogTrajectory.xyz", "w+")
for i in Qtraj:
    f.write("1\nLeapStep\n")
    f.write("H\t"+ str(i)+ "\t0\t0\n")
f.close()

In [124]:
def Euler(Z, forces, params, dt=0.1):
    """
    Inputs:
        Z: array of length 3*DoF with complex elements Zi = qi + i pi
        forces: Analytical forces as a function of the spatial coordinates q
        dt: optional time step
        params: Relevant information for forces
    Output:
        Updated Z values.
    """

    # Extract positions and momentums
    Z = np.array(Z)
    Q = Z.real
    P = Z.imag
    
    F = forces(Z, params)
    Qold = Q.copy()
    Pold = P.copy()
    
    # Masses are just 1
    Q = Qold + dt*P
    P = Pold + dt*F
    
    
    return(Q+1j*P)

def LeapFrog(Z, forces, params, dt=0.1):
    """
    Inputs:
        Z: 2D array of length 3*DoF with complex elements Zi = qi + i pi in each array
            Stores phase space coordinates at adjacent timesteps. Z = [Z_t, Z_{t-1]}]
        forces: Analytical forces as a function of the spatial coordinates q
        dt: optional time step
        params: Relevant information for forces
    Output:
        Updated Z values.
    """
    
    # Extract positions
    Qcurrent = Z[0].real
    Qlast = Z[1].real
    F = forces(Z[0], params)
    
    # Again just taking mass = 1
    Qnext = 2.0*Qcurrent - Qlast + 2.0*dt*dt*F
    
    # Keep in mind now im(all elements) = 0, so all we have are spatial coordinates. 
    Znew = np.array([Qnext, Qcurrent])
    return(Znew)
    
def forcesHarmonic(Z, params):
    """
    Inputs:
        Z: array of length 3*DoF with complex elements Zi = qi + i pi
        Params{"xo"}: Center of the harmonic potential
        Params{"SpringConstant"}: Spring constant, same for all qi.
        Params{"ComputePotential"}: Whether or not potential is to be computed
        Params{"PotentialFunction"}: If computing potential, pass function to do so.
    Outputs:
        F: Vector of dim(Z) with forces for the corresponding components.
    """
    xo = params["xo"]
    k = params["SpringConstant"]

    Q = Z.real
    F = np.zeros(len(Z))
    
    # Update elementwise the force on the Q[i]'th particle.
    for i, ele in enumerate(F):
        F[i] = -k*(Q[i]-xo)
    return(F)
    
def potentialHarmonic(Z, params):
    return()
    